In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from childes_mi.utils.paths import DATA_DIR, FIGURE_DIR, ensure_dir
from childes_mi.utils.general import flatten,save_fig

In [3]:
from childes_mi.information_theory import model_fitting as mf

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
MI_DF = pd.read_pickle(DATA_DIR / "mi" / "childes_mi_full_median_shuffled.pickle")

In [6]:
MI_DF[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_words,n_unique_words,top_10_words,median_n_words,shuffling
0,"[11.284785551820306, 11.07337340597912, 10.962...","[0.012132460827340517, 0.011977886679756525, 0...","[10.769425720743001, 10.740320650179003, 10.71...","[0.01151277952640151, 0.01159371678627942, 0.0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,1.5,15154,5439,NaN,38,full
1,"[11.17552222061179, 11.025854782677685, 10.947...","[0.011992495398434897, 0.011940992151928976, 0...","[10.770952584728056, 10.745712423437514, 10.72...","[0.011518514743174987, 0.011602685348349469, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,1.5,15154,5439,NaN,38,shuffle_within_utterances
2,"[11.066789609819926, 10.835337733067899, 10.77...","[0.011846333255002107, 0.011656988517723814, 0...","[10.782563066150653, 10.743549662551684, 10.73...","[0.01151692231709228, 0.011562181130191248, 0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,1.5,15154,5439,NaN,38,shuffle_between_utterances


In [7]:
MI_DF.shuffling.unique()

array(['full', 'shuffle_within_utterances', 'shuffle_between_utterances',
       'no_repeats', 'pos', 'shuffled_within_transcript'], dtype=object)

In [8]:
MI_DF = MI_DF.assign(**{i:np.nan for i in ['exp_results', 'pow_results', 'concat_results',
     'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
     'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak']})
MI_DF['curvature'] = MI_DF['curvature'].astype(object)

In [9]:
n = 100 # max distance for computation
for idx, row in tqdm(MI_DF.iterrows(), total=len(MI_DF)):
    # get signal
    sig = np.array(row.MI-row.shuff_MI)
    distances = row.distances
    sig = sig
    
    dist_mask = distances <= row.median_n_words
    distances = distances[dist_mask]
    sig = sig[dist_mask]
    
    # fit models
    results_power, results_exp, results_pow_exp, best_fit_model = mf.fit_models(distances, sig)
    
    # get fit results
    R2_exp, R2_concat, R2_power, AICc_exp, \
        AICc_pow, AICc_concat = mf.fit_results(sig, distances, 
                                              results_exp, results_power,
                                              results_pow_exp)
    
    
    # get model y
    distances_mod = np.logspace(0,np.log10(n), base=10, num=1000)
    if best_fit_model == 'pow_exp':
        y_model = mf.get_y(mf.pow_exp_decay, results_pow_exp, distances_mod)
    elif best_fit_model == 'exp':
        y_model = mf.get_y(mf.exp_decay, results_exp, distances_mod)
    elif best_fit_model == 'pow':
        y_model = mf.get_y(mf.powerlaw_decay, results_power, distances_mod)
    
    # get curvature of model_y
    curvature_model = mf.curvature(np.log(y_model))
    
    # if the best fit model is pow_exp, then grab the min peak
    if best_fit_model == 'pow_exp':
        # get peaks of curvature
        peaks = np.where((
            (curvature_model[:-1] < curvature_model[1:])[1:] & (curvature_model[1:] < curvature_model[:-1])[:-1]
        ))
        min_peak = peaks[0][0]
    else:
        min_peak = np.nan

    # get save model fit results to MI_DF
    MI_DF.loc[idx, np.array(['exp_results', 'pow_results', 'concat_results',
                         'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
                         'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak'])] = [
        results_exp, results_power, results_pow_exp,
        R2_exp, R2_concat, R2_power, AICc_exp,
        AICc_concat, AICc_pow, best_fit_model,
        curvature_model, min_peak
    ]

    print(row.age_cohort_low, row.age_cohort_high, best_fit_model, row.shuffling)

1 1.5 pow_exp full
1 1.5 pow_exp shuffle_within_utterances
1 1.5 pow shuffle_between_utterances
1 1.5 exp no_repeats
1 1.5 pow pos


/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/childes_mi/information_theory/model_fitting.py:8: RuntimeWarning: invalid value encountered in log
  return np.abs(np.log(y_true) - np.log(y_model)) * (1 / (np.log(1 + x)))


1 1.5 exp shuffled_within_transcript
1.5 2.0 pow_exp full
1.5 2.0 pow_exp shuffle_within_utterances
1.5 2.0 exp shuffle_between_utterances
1.5 2.0 pow_exp no_repeats
1.5 2.0 pow pos
1.5 2.0 pow shuffled_within_transcript
2 2.5 pow_exp full
2 2.5 pow_exp shuffle_within_utterances
2 2.5 exp shuffle_between_utterances
2 2.5 pow_exp no_repeats
2 2.5 pow_exp pos
2 2.5 pow shuffled_within_transcript
2.5 3.0 pow_exp full
2.5 3.0 pow_exp shuffle_within_utterances
2.5 3.0 exp shuffle_between_utterances
2.5 3.0 pow_exp no_repeats
2.5 3.0 pow_exp pos
2.5 3.0 exp shuffled_within_transcript
3 20.0 pow_exp full
3 20.0 pow_exp shuffle_within_utterances
3 20.0 pow_exp shuffle_between_utterances
3 20.0 pow_exp no_repeats
3 20.0 pow_exp pos
3 20.0 pow shuffled_within_transcript



In [10]:
MI_DF[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_words,n_unique_words,top_10_words,...,concat_results,R2_exp,R2_concat,R2_power,AICc_exp,AICc_concat,AICc_power,bestfitmodel,curvature,min_peak
0,"[11.284785551820306, 11.07337340597912, 10.962...","[0.012132460827340517, 0.011977886679756525, 0...","[10.769425720743001, 10.740320650179003, 10.71...","[0.01151277952640151, 0.01159371678627942, 0.0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,1.5,15154,5439,NaN,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.996648,0.997686,0.994729,-313.875806,-322.789288,-296.670201,pow_exp,"[3.503153918994611e-07, 5.186894218423023e-07,...",86.0
1,"[11.17552222061179, 11.025854782677685, 10.947...","[0.011992495398434897, 0.011940992151928976, 0...","[10.770952584728056, 10.745712423437514, 10.72...","[0.011518514743174987, 0.011602685348349469, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,1.5,15154,5439,NaN,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.992503,0.994776,0.991949,-298.095385,-306.654742,-295.386853,pow_exp,"[7.295804428129085e-06, 1.0914082349716316e-05...",198.0
2,"[11.066789609819926, 10.835337733067899, 10.77...","[0.011846333255002107, 0.011656988517723814, 0...","[10.782563066150653, 10.743549662551684, 10.73...","[0.01151692231709228, 0.011562181130191248, 0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1,1.5,15154,5439,NaN,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.941652,0.941667,0.941667,-136.960636,-131.801720,-136.970838,pow,"[2.0178313753163676e-06, 3.0352917363820564e-0...",NaN


In [13]:
MI_DF.columns

Index(['MI', 'MI_var', 'shuff_MI', 'shuff_MI_var', 'distances',
       'age_cohort_low', 'age_cohort_high', 'n_words', 'n_unique_words',
       'top_10_words', 'median_n_words', 'shuffling', 'exp_results',
       'pow_results', 'concat_results', 'R2_exp', 'R2_concat', 'R2_power',
       'AICc_exp', 'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature',
       'min_peak'],
      dtype='object')

In [ ]:
parameter_translation = {
    "e_init": "a",
    "e_decay_const": "b",
    "p_init": "c",
    "p_decay_const": "d",
    "intercept": "f",
}

In [52]:
param_df = pd.DataFrame(
    columns=[
        "dataset", 
        "age_low", 
        "age_high", 
        "a_value", 
        "a_stderr",
        "b_value", 
        "b_stderr",
        "c_value", 
        "c_stderr",
        "d_value", 
        "d_stderr",
        "f_value", 
        "f_stderr"
    ]
)
for idx, row in MI_DF[MI_DF.shuffling == 'full'].iterrows():
    param_vals = {}
    param_stderrs = {}
    for param in row.concat_results.params:
        param_vals[param] = row.concat_results.params[param].value
        param_stderrs[param] = row.concat_results.params[param].stderr
    param_df.loc[len(param_df)] = [
        'childes',
        row.age_cohort_low,
        row.age_cohort_high,
        param_vals["e_init"],
        param_stderrs["e_init"],
        param_vals["e_decay_const"],
        param_stderrs["e_decay_const"],
        param_vals["p_init"],
        param_stderrs["p_init"],
        param_vals["p_decay_const"],
        param_stderrs["p_decay_const"],
        param_vals["intercept"],
        param_stderrs["intercept"],

    ]

In [53]:
param_df

,dataset,age_low,age_high,a_value,a_stderr,b_value,b_stderr,c_value,c_stderr,d_value,d_stderr,f_value,f_stderr
0,childes,1,1.5,0.386827,0.100693,0.645071,0.113230,0.145095,0.038388,-1.382267,0.345356,0.167552,0.002869
1,childes,1.5,2.0,0.194101,0.022386,0.382302,0.034353,0.283355,0.015715,-1.461465,0.083040,0.057093,0.000598
2,childes,2,2.5,0.184505,0.021723,0.418270,0.032997,0.345893,0.013929,-1.463729,0.040374,0.039624,0.000243
3,childes,2.5,3.0,0.239060,0.098850,0.753414,0.104630,0.391075,0.038772,-1.367054,0.053455,0.027329,0.000348
4,childes,3,20.0,0.639437,0.064763,1.081856,0.046559,0.222734,0.021559,-1.238223,0.041301,0.008262,0.000081


In [54]:
ensure_dir(DATA_DIR / 'param_dfs')
param_df.to_pickle(DATA_DIR / 'param_dfs' / 'childes.pickle')